In [1]:
from qdrant_client import QdrantClient, models

In [2]:
qd_client = QdrantClient("http://localhost:6333")

In [3]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [22]:
from fastembed import TextEmbedding

In [24]:
embedding_model = TextEmbedding(model_name=model_handle)

Question 1

In [41]:
documents: list[str] = [
    "I just discovered the course. Can I join now?",
]
embeddings = list(embedding_model.embed(documents))

In [38]:
min(embeddings[0])

np.float64(-0.11726373885183883)

In [44]:
import numpy as np
np.linalg.norm(embeddings[0])

np.float64(1.0)

In [46]:
embeddings[0].dot(embeddings[0])

np.float64(1.0000000000000002)

Question 2

In [47]:
documents: list[str] = [
    "I just discovered the course. Can I join now?",
    "Can I still join the course after the start date?"
]
embeddings = list(embedding_model.embed(documents))

In [48]:
q = embeddings[0]
q2 = embeddings[1]

q.dot(q2)

np.float64(0.9008528895674548)

Question 3

In [65]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [72]:
questions: list[str] = [
    "I just discovered the course. Can I join now?"
]
embedding_question = list(embedding_model.embed(questions))

In [66]:
texts = [item["text"] for item in documents]
embeddings = list(embedding_model.embed(texts))

In [77]:
index = 0
res = -1
temp = 0
for emb in embeddings:
    dot = emb.dot(embedding_question[0])
    print(dot)
    print(index)
    if (dot > temp):
        temp = dot
        res = index
    index = index + 1

0.7629684696540238
0
0.8182378150042889
1
0.8085397398734189
2
0.7133079015686243
3
0.7304499234333598
4


In [76]:
res

1

Question 4

In [78]:
texts = ["{} {}".format(item["question"], item["text"]) for item in documents]
embeddings = list(embedding_model.embed(texts))

In [79]:
index = 0
res = -1
temp = 0
for emb in embeddings:
    dot = emb.dot(embedding_question[0])
    print(dot)
    print(index)
    if (dot > temp):
        temp = dot
        res = index
    index = index + 1

0.8514543236908068
0
0.8436594159113068
1
0.8408287048502558
2
0.7755157969663908
3
0.8086007795043938
4


In [80]:
res

0

Question 5

In [82]:
min_dim = min(TextEmbedding.list_supported_models(), key=lambda x: x["dim"])

In [83]:
min_dim

{'model': 'BAAI/bge-small-en',
 'sources': {'hf': 'Qdrant/bge-small-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.13,
 'additional_files': [],
 'dim': 384,
 'tasks': {}}

Question 6

In [84]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [90]:
collection_name = "zoomcamp-vector-homework"

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [91]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model="BAAI/bge-small-en")
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [92]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)   

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [95]:
def vector_search(question):
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model="BAAI/bge-small-en" 
        ),
        limit=5,
        with_payload=True
    )
    
    return query_points.points

In [96]:
vector_search("I just discovered the course. Can I join now?")

[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=6, version=0, score=0.86918855, payload={'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'section': 'General course-related questions', 'question': 'How long is the course?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=No